In [13]:
import pandas as pd
import numpy as np
import h5py
import math

In [8]:
def get_attributes_from_hdf_5 (h5_path, part_name):
    attrs_dict = {}
    with h5py.File(h5_path,'r') as h5:
        for item in h5[part_name].attrs.keys():
            attrs_dict[item] = h5[part_name].attrs[item]
        return attrs_dict

In [17]:
def calculate_part_dimensions(h5_path, part_name):
    #here one should additionally check whether all the required keys are in the dictionary
    dict = {}
    attrs_dict = get_attributes_from_hdf_5(h5_path, part_name)
    maxX = attrs_dict.get('maxX')
    maxY = attrs_dict.get('maxY')
    maxZ = attrs_dict.get('maxZ')

    minX = attrs_dict.get('minX')
    minY = attrs_dict.get('minY')
    minZ = attrs_dict.get('minZ')

    layerThickness = attrs_dict.get('layerThickness')
    height = abs(maxZ-minZ)

    dict['lengthX'] = abs(maxX-minX)
    dict['lengthY'] = abs(maxY-minY)
    dict['height'] = abs(maxZ-minZ)
    dict['minX'] = minX
    dict['minY'] = minY

    dict['number_of_layers'] = int(height/layerThickness)

    return dict

In [18]:
calculate_part_dimensions('/home/jan/Documents/CodeTDMStoHDF/Ausgangsdaten/examplerRun.h5','0_00003_Canti3_cls')

{'lengthX': 36000,
 'lengthY': 6000,
 'height': 4475,
 'minX': -113486,
 'minY': 37743,
 'number_of_layers': 179}

In [19]:
dimension_dict = calculate_part_dimensions('/home/jan/Documents/CodeTDMStoHDF/Ausgangsdaten/examplerRun.h5','0_00003_Canti3_cls')
length_x_part = dimension_dict['lengthX']
length_y_part = dimension_dict['lengthY']
number_of_layers_part = dimension_dict['number_of_layers']
voxel_size = 100
num_voxel_layers = 20

num_voxels = get_number_voxel(length_x_part, length_y_part, number_of_layers_part, voxel_size, num_voxel_layers)
num_voxels_x = num_voxels[0]
num_voxels_y = num_voxels[1]
num_voxels_z = num_voxels[2]

In [20]:
num_voxels

(360, 60, 9)

In [21]:
def get_2D_data_from_h5(h5_path, part_name, Slice_name):
    with h5py.File(h5_path,'r') as h5:
        X_Axis = h5[part_name][Slice_name]['X-Axis']
        Y_Axis = h5[part_name][Slice_name]['Y-Axis']
        Area = h5[part_name][Slice_name]['Area']
        Intensity = h5[part_name][Slice_name]['Intensity']

        help_arr = np.column_stack((X_Axis, Y_Axis, Area, Intensity))
        df = pd.DataFrame(help_arr, columns=['x','y','area','intensity'])
        return df

In [39]:
df = get_2D_data_from_h5('/home/jan/Documents/CodeTDMStoHDF/Ausgangsdaten/examplerRun.h5','0_00003_Canti3_cls', 'Slice00001')

In [40]:
minX = df['x'].min()
minY = df['y'].min()
print(minX)
print(minY)

6039.0
-28561.0


In [41]:
def dock_df_to_zero(df, minX, minY):
    if minX >= 0 and minY >=0:
        print('fall1')
        df['x'] = df['x'] - minX
        df['y'] = df['y'] - minY
    elif minX < 0 and minY <0:
        print('fall2')
        df['x'] = df['x'] + abs(minX)
        df['y'] = df['y'] + abs(minY)
    elif minX >= 0 and minY <0:
        print('fall3')
        df['x'] = df['x'] - minX
        df['y'] = df['y'] + abs(minY)
    elif minX < 0 and min >= 0:
        print('fall4')
        df['x'] = df['x'] + abs(minX)
        df['y'] = df['y'] - minY
    return df

In [42]:
new_df = dock_df_to_zero(df, minX, minY)

fall3


In [43]:
new_df

,x,y,area,intensity
0,0.0,8977.0,0.0,0.0
1,0.0,8977.0,0.0,0.0
2,0.0,8977.0,0.0,0.0
3,0.0,8977.0,0.0,0.0
4,0.0,8977.0,0.0,0.0
...,...,...,...,...
143196,3528.0,8977.0,0.0,0.0
143197,3528.0,8977.0,0.0,0.0
143198,3528.0,8977.0,0.0,0.0
143199,3528.0,8977.0,0.0,0.0
